# Dynamic Attributes and Properties
Data attributes and methods are collectively known as attributes in Python: a method is just an attribute that is callable. Besides data attributes and methods, we can also create properties, which can be used to replace a public data attribute with accessor methods (i.e., getter/setter), without changing the class interface. This agrees with the Uniform access principle: All services offered by a module should be available through a uniform notation, which does not betray whether they are implemented through storage or through computation.

Besides properties, Python provides a rich API for controlling attribute access and implementing dynamic attributes. The interpreter calls special methods such as __getattr__ and __setattr__ to evaluate attribute access using dot notation.

In [7]:
class A:
    def __getattr__(self, attr):
        ''' __getattr__ is called when the default attribute access fails with an AttributeError.
            It lets you define an optional value to return if the attribute doesn't exist.
            Similar to dict.get()
        '''
        print("Getting attribute ", attr)
        return 42


In [8]:
a = A()
a.x

Getting attribute  x


42

In [10]:
d = {"a": 1, "b":2}
d.get("c", 42)

42

In [9]:
class A:
    def __init__(self):
        self.x = 7

    def __setattr__(self, attr, value):
        ''' Called when an attribute assignment is attempted. '''
        print("Setting attribute ", attr, " to ", value)
        self.__dict__[attr] = value


In [10]:
a = A()  # setattr called on init

Setting attribute  x  to  7


In [11]:
a.x

7

In [12]:
a.x = 9  # also when we set here

Setting attribute  x  to  9


We could stop someone from setting attributes that begin with an underscore...

In [13]:
class A:
    def __init__(self):
        self._x = 7
        self._y = 8
        
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __setattr__(self, attr, value):
        if attr in self.__dict__:
            if attr.startswith("_"):
                print("Nah bro!")
                return
        self.__dict__[attr] = value


In [14]:
a = A()
a.x, a.y

(7, 8)

In [15]:
a._x = 12

Nah bro!


In [16]:
a.x

7

## Data Wrangling with Dynamic Attributes

In [18]:
d = { "Schedule":
 { "conferences": [{"serial": 115 }],
 "events": [
 { "serial": 34505,
 "name": "Why Schools Don´t Use Open Source to Teach Programming",
 "event_type": "40-minute conference session",
 "time_start": "2014-07-23 11:30:00",
 "time_stop": "2014-07-23 12:10:00",
 "venue_serial": 1462,
 "description": "Aside from the fact that high school programming...",
 "website_url": "http://oscon.com/oscon2014/public/schedule/detail/34505",
 "speakers": [157509],
 "categories": ["Education"] }
 ],
 "speakers": [
 { "serial": 157509,
 "name": "Robert Lefkowitz",
 "photo": None,
 "url": "http://sharewave.com/",
 "position": "CTO",
 "affiliation": "Sharewave",
 "twitter": "sharewaveteam",
 "bio": "Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup..." }
 ],
 "venues": [
 { "serial": 1462,
 "name": "F151",
 "category": "Conference Venues" }
 ]
 }
}


In [19]:
d

{'Schedule': {'conferences': [{'serial': 115}],
  'events': [{'serial': 34505,
    'name': 'Why Schools Don´t Use Open Source to Teach Programming',
    'event_type': '40-minute conference session',
    'time_start': '2014-07-23 11:30:00',
    'time_stop': '2014-07-23 12:10:00',
    'venue_serial': 1462,
    'description': 'Aside from the fact that high school programming...',
    'website_url': 'http://oscon.com/oscon2014/public/schedule/detail/34505',
    'speakers': [157509],
    'categories': ['Education']}],
  'speakers': [{'serial': 157509,
    'name': 'Robert Lefkowitz',
    'photo': None,
    'url': 'http://sharewave.com/',
    'position': 'CTO',
    'affiliation': 'Sharewave',
    'twitter': 'sharewaveteam',
    'bio': 'Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup...'}],
  'venues': [{'serial': 1462,
    'name': 'F151',
    'category': 'Conference Venues'}]}}

In [3]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

if os.path.isfile(JSON):
    os.remove(JSON)

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            return remote.read()


In [4]:
feed = load()

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: downloading http://www.oreilly.com/pub/sc/osconfeed to data/osconfeed.json
  from ipykernel import kernelapp as app


In [7]:
len(feed)

787465

## Using a Property for Attribute Validation